# Versión 2
## Avances


## Conclusiones


In [2]:
#LIBRERÍAS

#Librerías de Sistema
import os
import sys

#Librerías de Manejo de Datos
import pandas as pd
import numpy as np

#Librerías de Interfaz
from tkinter import filedialog
from tkinter import *
from tkinter import messagebox

#Librerías para EEG
import mne

### Variables Globales 

In [3]:
usecols = 0   #definimos las variables de antemano para que las use todo el algoritmo, como globales
skiprows = 0
sep = ","
col_names = []
col_types = []

chan_quant = 0

chan_pos = []
chan_selecc = [] 

### Funciones

In [ ]:
#we create the function get indexes
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]

#This would work like : X = ["Mario", "Luigi", "Yoshi", "Mario"]
#print(get_indexes("Mario", X)) -> Out: [0,3]
#print(get_indexes("Luigi", X)) -> Out: [1]

In [4]:
def yn_quest(quest):
    while True:
        answ = input(quest + ' [Y/N]: ')
        if answ == 'Y' or answ == 'y' or answ == 'N' or answ == 'n':
            break
        else:
            print('La respuesta ingresada no es correcta. Ingrese nuevamente.')
    return answ

In [5]:
def load_signal():   
    nom_s = input("¿Qué nombre tendrá la señal?: ")

    while True:
        root = Tk() #Crea la ventana root
        xs = filedialog.askopenfilename(title = "Seleccione archivo de señal",          #Dice que va a sacar la variable Xs como el filename de un archivo
                                        filetypes = (("Documento de texto","*.txt"),
                                                     ("Excel (post 2003)", ".xlsx"),
                                                     ("Excel (prev 2003)", ".xls"),
                                                     ("Todos los archivos","*.*")
                                                    ) #Con titulo de ventana y opciones de abrir archivos de ese tipo
                                       ) 
        root.mainloop() #esto es para que la ventana quede abierta hasta que se de una respuesta, en este caso la selección de un archivo
        
        print (xs)

        if ".txt" in xs:
            opt = yn_quest('Se trata de un archivo exportado por el Software de OpenBCI?')
            if opt == 'Y' or opt == 'y':
                XS = openbci_topd(xs)
            else:
                XS = csv_topd(xs)
            print("Señal Cargada. Formato TXT.")
            break
        
        elif ".xlsx" in xs or ".xls" in xs:
            XS = excel_topd(xs, interval = False)
            print("Señal Cargada. Formato Excel.") #Esto sería reemplazado por la función "leer txt" que tomaría la dirección y parámetros como Cantidad de columnas, Nombres de columnas, etc (o se puede probar con la direccion sola y si no funciona pedir parametros)
            break

        elif xs == "":
            print("Operación Cancelada.")
            break

        else:
            print("Formato no admitido. Intente de Nuevo.")
            break

    return XS, nom_s

In [6]:
def csv_topd(truepath):
    global usecols
    global skiprows
    global sep
    global col_names
    global col_types
    
    while True:
        col_rows_sep()
        
        save_opt = input('Desea guardar esta configuración?: ') #Completar, cómo y dónde y la carga de la configuracion
        
        try:
            X = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, names=col_names, usecols=range(usecols), lineterminator = '\n')
        except:
            print('\n\nLos datos ingresados no encajan con el archivo cargado, reintentar.\n\n')
            
        else:
            break
    
    X.col_names = col_names #Esto sería medio de gusto si ahora col_names y col_types son globales
    X.col_types = col_types
    
    return X

In [7]:
def openbci_topd(truepath):
    global usecols
    global skiprows
    global sep
    global col_names
    global col_types
    
    selecc_predet = yn_quest('¿Usará la configuración predeterminada de OpenBCI?')
    
    if selecc_predet == 'N' or selecc_predet == 'n': #si decide que no el usuario ingresa los datos a tener en cuenta al leer, pero un archivo txt de OpenBCI
        col_rows_sep()
        
        save_opt = input('Desea guardar esta configuración?: ') #Completar, cómo y dónde y la carga de la configuracion
        
    elif selecc_predet == 'Y' or selecc_predet == 'y':
        sep = ","
        col_names = ['Sample', 'Ch1', 'Ch2', 'Ch3', 'Ch4', 'x', 'y', 'z', 'Time', ' ']
        col_types = ['Misc', 'Channel', 'Channel', 'Channel', 'Channel', 'Accel', 'Accel', 'Accel', 'Time', 'Misc'] #Esto sirve para saber que cantidad de columnas son canales, y que canal tiene los marcadores
        usecols = 10
        skiprows = 6
        
    while True:        
        try:           
            X = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, lineterminator = '\n')

            assert X.values[0,0] == 1 

        except AssertionError:   
            skiprows = int(input("El formato seleccionado no encaja en el archivo presentado. ¿Cuántas filas deben saltearse antes de comenzar a leer datos?: "))

            Xt = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, lineterminator = '\n')
            if all(X.values[0:4,0] != [0,1,2,3]):
                sep = ";"
                Xt = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, lineterminator = '\n')

            if all(X.values[0:4,0] != [0,1,2,3]):
                sep = "."
                Xt = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, lineterminator = '\n')

            if all(X.values[0:4,0] != [0,1,2,3]):
                sep = input("El formato seleccionado no encaja en el archivo presentado. Ingresar como se separan las filas en el archivo CSV (.txt): ")
#Faltaría un contador que vuelva hacia atrás si aún ingresando el formato manualmente sigue sin encajar,porque talvez no tiene la columna samples, podria aprovechar el return
        else:
            break

    #vamos a hacer dos whiles infinitos, para que en el primero no se preocupe por los nombres y cantidad de columnas, sino por cuantas filas saltear, y en este próximo vea columnas

    while True:        
        try:           
            X = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, names=col_names, usecols=range(usecols), lineterminator = '\n')
        except:                   
            usecols = int(input("El formato seleccionado no encaja en el archivo presentado. ¿Cuántas columnas posee el archivo? : "))
            col_names = []
            col_types = []
            for i in range(usecols):
                print("Nombre de la columna número ", (i+1))
                nombre = input("Ingresar: ")
                print("Tipo de la columna número ", (i+1), " [Channel, Time, Accel, Markers, Misc]") #Si no se puede armar la seleccion con Tkinter, buscar la librería pick de wong2
                tipo = input("Ingresar: ")
                col_names.append(nombre)
                col_types.append(tipo)
        else:                     
            break 
    
    return X

In [8]:
def excel_topd(truepath, interval = True):
    global usecols
    global skiprows
    global col_names
    global col_types
    
    while True:
        if interval == True:
            
            header = 0
            index_col = 0
            usecols = 3
            skiprows = 0
            col_names = ['Name', 'Startpoint']

            endpoint_opt = yn_quest("Does it has an endpoint column?")
            if endpoint_opt == 'Y' or endpoint_opt == 'y':
                col_names.append('Endpoint')
                usecols =+ 1

            duration_opt = yn_quest("Does it has a duration column?")
            if duration_opt == 'Y' or duration_opt == 'y':
                col_names.append('Duration')
                usecols =+ 1

        elif interval == False:
            header = None
            index_col = None
            col_rows_sep()
        
        try:
            X = pd.read_excel(truepath, usecols = range(usecols), skiprows = skiprows, names = col_names, header=header, index_col=index_col)
        except:
            print('\n\nLos datos ingresados no encajan con el archivo cargado, reintentar.\n\n')
        else:
            break
    
    return X

In [9]:
def col_rows_sep():
    
    def print_col():
        col_names_win.destroy()
        global col_names
        global col_types
        
        col_names = []
        col_types = []
        for i in range(columnas.get()):
            col_names.append(col_n[i].get())
            col_types.append(col_t[i].get())
                    
    def selec_pd ():       
        
        conf = messagebox.askquestion("Confirm","Will you select : " + str(columnas.get()) + " columns & " + str(filas.get()) + " rows?")
        if conf == "yes":
            global usecols   # cada vez que las queremos modificar tenemos que decirle que nos referimos a las globales
            global skiprows  #Deberíamos buscar una solución para esto, ya que no es muy bueno, pero no podemos retornar
            global sep        #correctametne las variables porque estan en una función anidada
            usecols = columnas.get()   
            skiprows = filas.get()     
            sep = separador.get()
                        
            selec_pd_win.destroy()
            global col_names_win 
            col_names_win = Tk()
            col_names_win.title("Columns")
            col_names_win.label1 = Label(col_names_win, text = "Name:").place(x=30,y=20)
            col_names_win.label2 = Label(col_names_win, text = "Type:").place(x=170,y=20)
        
            col_names_win.geometry("350x"+str(40+columnas.get()*35))
            for i in range(columnas.get()):
                col_n.append(StringVar())
                col_t.append(StringVar())
                col_names_win.entry = Entry(col_names_win, textvariable=col_n[i]).place(x=30 , y=40+i*26)
                col_names_win.spinbox = Spinbox(col_names_win, values = ("Channel", "Misc", "Tag", "Time", "Accel"), wrap =True, textvariable =col_t[i]).place(x=170 , y=40+i*26)
            col_names_win.button1 = Button (col_names_win , text="Print Values", command=print_col).place(x=100, y=40+26*columnas.get()) #command= imprimir)
            col_names_win.mainloop()

    selec_pd_win = Tk() #Crea la ventana
    
    col_n = []
    col_t = []
    
    columnas = IntVar()
    filas = IntVar()
    separador = StringVar()

    selec_pd_win.title("Set parameters for Pandas" ) 
    selec_pd_win.geometry("350x200")

    selec_pd_win.label1 = Label(selec_pd_win , text= "Rows to Skip:").place(x=30,y=20)
    selec_pd_win.entry1 = Entry(selec_pd_win, textvariable=filas).place(x=170 , y=20)

    selec_pd_win.label2 = Label(selec_pd_win, text="Columns to use:").place(x=30 , y=60)
    selec_pd_win.entry2 = Entry(selec_pd_win, textvariable=columnas).place(x=170 , y=60)

    selec_pd_win.label3 = Label(selec_pd_win, text="Separator to use:").place(x=30 , y=100)
    selec_pd_win.entry3 = Entry(selec_pd_win, textvariable=separador).place(x=170 , y=100)

    selec_pd_win.button1 = Button (selec_pd_win , text="Accept", command=selec_pd).place(x=170 , y=150) #command= imprimir)

    selec_pd_win.mainloop()

In [10]:
def chan_config(col_names, col_types):
    
    def print_chan():
        global chan_pos
        global chan_selecc
        chan_pos = []
        chan_selecc = []
        
        chan_selec_win.destroy()
        for i in range(chan_quant):
            chan_pos.append(chan_p[i].get())
            chan_selecc.append(chan_s[i].get())
    
    global chan_quant
    chan_quant = 0
    chan_all = []
    chan_p = []
    chan_s = []
    
    i = 0
    for word in col_types:
        if word == 'Channel':
            chan_all.append(col_names[i])
            chan_quant += 1
        i += 1

    chan_selec_win = Tk()
    chan_selec_win.title("Channel Selection")
    chan_selec_win.label1 = Label(chan_selec_win, text = str(chan_quant) +' Channels were detected, wich ones will you use?').place(x=30,y=10)

    chan_selec_win.geometry("500x"+str(80+chan_quant*35))
    for i in range(chan_quant):
        chan_s.append(IntVar())
        chan_p.append(StringVar())

        chan_selec_win.label2 = Label(chan_selec_win, text = chan_all[i] + " column:").place(x=20 , y=40+i*26)
        chan_selec_win.check1 = Checkbutton(chan_selec_win, text = "Import?", variable=chan_s[i], onvalue=1, offvalue=0).place(x=90 , y=40+i*26)
        chan_selec_win.label3 = Label(chan_selec_win, text = "Channel Position:").place(x=180 , y=40+i*26)
        chan_selec_win.entry1 = Entry(chan_selec_win, textvariable = chan_p[i]).place(x=300 , y=40+i*26)

    chan_selec_win.button1 = Button (chan_selec_win , text="Print Value", command=print_chan).place(x=100, y=60+26*chan_quant) #command= imprimir)

    chan_selec_win.mainloop()

In [33]:
def pd_toarray(X_pd):
    
    i = 0
    j = 0
    chan_list = []
    accel_list = []
    tag_list = []
    global col_types
    global chan_selecc
    
    while i < X_pd.shape[-1]:
        if col_types[i] == "Channel":
            if chan_selecc[j] == 1:
                chan_list.append(X_pd.columns[i])
            j += 1

        if col_types[i] == " Accel":
            accel_list.append(X_pd.columns[i])

        if col_types[i] == "Tag":
            tag_list.append(X_pd.columns[i])
        i += 1

    X_chan = (X_pd.filter(chan_list)).values
    X_accel = (X_pd.filter(accel_list)).values
    X_tag = (X_pd.filter(tag_list)).values
    
    return X_chan, X_accel, X_tag

In [59]:
def array_tomneRaw(X_chan):
    global chan_selecc
    global chan_pos
    ch_names = []
    #montage = []

    for i in get_indexes(1,chan_selecc):
        ch_names.append('Ch'+ str(i+1))
        #montage.append(chan_pos[i])   No funciona así, porque hay que darle una variable montage especifica, hay que ver como crearla

    X_mneinfo = mne.create_info(ch_names = ch_names, 
                                ch_types = 'eeg', 
                                #montage = montage,  ver como crear el montage https://martinos.org/mne/dev/generated/mne.channels.Montage.html#mne.channels.Montage
                                sfreq = 200) #Cambiar por fq_sub

    X_mneRaw = mne.io.RawArray(np.transpose(X_chan), info = X_mneinfo)
    
    return X_mneRaw #ver si también retornamos X_mneinfo


### Cuerpo del Código

In [61]:
X_pd, signalname = load_signal()

chan_config(col_names, col_types)

X_chan, X_accel, X_tag = pd_toarray(X_pd)

X_mneRaw = array_tomneRaw(X_chan)

¿Qué nombre tendrá la señal?: Agus
C:/Users/Mari y Juanma/Desktop/Nerdeala/OTTAA/Comparacion/96cb-Fase1-AR1-18-01-19.txt
Se trata de un archivo exportado por el Software de OpenBCI? [Y/N]: Y
¿Usará la configuración predeterminada de OpenBCI? [Y/N]: Y
Señal Cargada. Formato TXT.
Creating RawArray with float64 data, n_channels=3, n_times=320697
    Range : 0 ... 320696 =      0.000 ...  1603.480 secs
Ready.


In [69]:
X_mneRaw

<RawArray  |  None, n_channels x n_times : 3 x 320697 (1603.5 sec), ~7.4 MB, data loaded>